In [1]:
import pandas as pd
import os
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms
from torch.utils.data import DataLoader

%matplotlib inline

from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet import preprocess_input as resnet_preprocessing_input
import torch

## Submission notebook

- weights after training on each folder separately were saved
- apply classification pretrained model on submission dataset
- depending on the class - predict the score of each image using corresponding weights for prediction

In [2]:
# important: It should be defined first, pytorch need to specify device for every tensor
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:0


In [3]:
from torchvision import transforms

image_size = 299

# simple transfrom for input X
transform = transforms.Compose([
    transforms.Resize(image_size), 
    transforms.CenterCrop(image_size),
    # require to be tensor for the input of neural net such as resnet18
    transforms.ToTensor(),
    # commonly used normalizer to transfrom 0-255 -> #TODO
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

# transform from 0-100 to 0-1 for label y
def target_transform(y):
    return y/100

def target_inverse_transform(y):
    return y*100

In [4]:
model_conv_dog=torch.load('../input/pawpularity2/resnet18_dog')
model_conv_cat=torch.load('../input/pawpularity2/resnet18_cat')
model_conv_smth=torch.load('../input/pawpularity2/resnet18_smth')
ResNet50_model=tf.keras.models.load_model('../input/pawpularity2/ResNet50_model')

2022-02-03 10:43:27.306535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-03 10:43:27.307575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-03 10:43:27.308212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-03 10:43:27.309000: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
test_dir = '../input/petfinder-pawpularity-score/test'
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
result = []
predictor = nn.Sigmoid()
for image_id in test_df.iloc[:, 0]:
    img_path = os.path.join(test_dir, image_id)
    image = Image.open(img_path + '.jpg') # read_image
    img=image.resize((224, 224), Image.ANTIALIAS)
    converted_image = img_to_array(img)
    reshaped_image = converted_image.reshape((1, converted_image.shape[0], converted_image.shape[1], converted_image.shape[2]))
    resnet_preprocessed_image = resnet_preprocessing_input(reshaped_image)
    probs= ResNet50_model.predict(resnet_preprocessed_image)
    position_of_max = np.argmax(probs)
    #print(position_of_max)
    if ((position_of_max >= 151) & (position_of_max <= 268)):
        transformed = transform(image)
        transformed = transformed.reshape(1, transformed.shape[0], transformed.shape[1], transformed.shape[2]).to(device)
        output=model_conv_dog(transformed)        
    elif ((position_of_max >= 281) & (position_of_max <= 285)):
        transformed = transform(image)
        transformed = transformed.reshape(1, transformed.shape[0], transformed.shape[1], transformed.shape[2]).to(device)
        output = model_conv_cat(transformed)
    else:
        #print('i')
        transformed = transform(image)
        transformed = transformed.reshape(1, transformed.shape[0], transformed.shape[1], transformed.shape[2]).to(device)
        #print(transformed.shape)
        output = model_conv_dog(transformed)
    pred = target_inverse_transform(predictor(output))
    pred=pred.cpu()
    pred=pred.detach().numpy()
    pred=pred[0][0]
    #print(pred[0][0])
    #print(pred.type)
    result.append(pred)
    #print(result)
        
result_df = pd.DataFrame({'Id': test_df.iloc[:, 0], 'Pawpularity': result})
#print(result_df)
result_df.to_csv( 'submission.csv', index=False)

2022-02-03 10:43:37.158669: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-03 10:43:38.769666: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [6]:
#pd.read_csv('submission.csv')